
#  GPT-2 Fine-Tuning with and without PEFT (LoRA) + CodeCarbon

In this notebook you will find:

-  Full fine-tuning (no PEFT)
-  LoRA / PEFT fine-tuning
-  Carbon emissions tracking for both runs using **CodeCarbon**




In [16]:

# Install required libraries (run once per session)
!pip install -q transformers datasets peft codecarbon accelerate


In [17]:

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
)
from peft import LoraConfig, get_peft_model
from codecarbon import EmissionsTracker

# Base configuration
MODEL_ID = "ZigZeug/gpt2-finetuned-base"
DATASET_NAME = "flytech/python-codes-25k"
MAX_SAMPLES = 12500

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Load dataset
raw_dataset = load_dataset(DATASET_NAME, split="train")
raw_dataset = raw_dataset.shuffle(seed=42).select(range(MAX_SAMPLES))

system_message = "You are an AI assistant specialized in generating Python code.{schema}"

def to_chat_text(sample):
    """Turn one JSON instruction sample into a single training string."""
    system = system_message.format(schema=sample["instruction"])
    user = f"{sample['instruction']}\n{sample['input']}"
    assistant = sample["output"]
    text = (
        f"System: {system}\n\n"
        f"User: {user}\n\n"
        f"Assistant: {assistant}\n"
    )
    return {"text": text}

dataset_text = raw_dataset.map(to_chat_text, remove_columns=raw_dataset.column_names)
print(dataset_text[0]["text"][:500])
print("Number of training samples:", len(dataset_text))


Using device: cuda
System: You are an AI assistant specialized in generating Python code.Write a Python program to generate a sorted list of unique numbers from 0 to 9 in random order

User: Write a Python program to generate a sorted list of unique numbers from 0 to 9 in random order


Assistant: ```python
import random

# generating a list of unique numbers from 0 to 9 in random order
random_numbers = random.sample(range(0, 10), 10)

# sort list of numbers 
random_numbers.sort()

# print sorted list of random nu
Number of training samples: 12500


In [18]:

# Tokenizer and tokenization

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

MAX_LENGTH = 512

def tokenize_function(batch):
    outputs = tokenizer(
        batch["text"],
        truncation=True,
        max_length=MAX_LENGTH,
        padding=False,
    )
    # For causal LM, labels are the same as input_ids
    outputs["labels"] = outputs["input_ids"].copy()
    return outputs

tokenized_dataset = dataset_text.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"],
)

# Simple train/eval split (optional)
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

print(train_dataset[0])


{'input_ids': [11964, 25, 921, 389, 281, 9552, 8796, 16976, 287, 15453, 11361, 2438, 13, 8645, 378, 257, 1351, 286, 1936, 3835, 3519, 284, 4572, 4673, 198, 198, 12982, 25, 2980, 378, 257, 1351, 286, 1936, 3835, 3519, 284, 4572, 4673, 628, 198, 48902, 25, 3423, 318, 257, 1351, 286, 1936, 4047, 7151, 3835, 3519, 284, 4572, 4673, 25, 198, 198, 16, 13, 5633, 47546, 31517, 653, 290, 10850, 18252, 30, 416, 12803, 337, 13, 16559, 25, 770, 1492, 10969, 257, 9815, 9793, 284, 262, 4755, 10233, 287, 3912, 9465, 290, 4572, 4673, 13, 198, 17, 13, 5633, 464, 40531, 12, 9876, 10850, 18252, 4897, 30, 416, 843, 380, 88, 5481, 21862, 25, 770, 1492, 3769, 257, 35327, 16700, 286, 262, 749, 1593, 10838, 290, 7605, 287, 4572, 4673, 13, 198, 18, 13, 5633, 37906, 10850, 18252, 30, 416, 26190, 28513, 354, 4914, 25, 770, 1492, 13692, 319, 1262, 11361, 284, 3494, 4572, 4673, 16113, 290, 7605, 13, 198, 19, 13, 5633, 39, 1746, 12, 2202, 10850, 18252, 351, 10286, 15813, 12, 20238, 290, 309, 22854, 37535, 30, 416, 1

In [19]:

# LoRA / PEFT configuration

peft_config = LoraConfig(
    r=256,
    lora_alpha=128,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

#  Base TrainingArguments (shared)
base_training_args = dict(
    num_train_epochs=1,                  # increase later if needed
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
    fp16=torch.cuda.is_available(),
)


In [20]:
from huggingface_hub import login
login()


In [23]:
os.makedirs("codecarbon_logs", exist_ok=True)

def train_with_emissions(use_peft: bool):
    exp_name = "full_finetune" if not use_peft else "peft_lora"
    print("\n==============================")
    print(f" Starting experiment: {exp_name}")
    print("==============================")

    tracker = EmissionsTracker(
        project_name=f"gpt2_{exp_name}",
        output_dir="codecarbon_logs",
        log_level="error",
        save_to_file=True,
    )
    tracker.start()

    model = AutoModelForCausalLM.from_pretrained(MODEL_ID)
    model.to(device)

    if use_peft:
        model = get_peft_model(model, peft_config)
        model.print_trainable_parameters()

    output_dir = f"outputs_{exp_name}"

    training_args = TrainingArguments(
        output_dir=output_dir,
        push_to_hub=True,
        **base_training_args,
    )

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
    )

    trainer.train()


    trainer.push_to_hub(
        commit_message=f"Upload final {exp_name} model"
    )

    tokenizer.push_to_hub(
        repo_id=f"ngbinetou/{exp_name}",
        commit_message=f"Upload tokenizer for {exp_name}"
    )


    emissions = tracker.stop()
    print(f"Training finished: {exp_name}")
    print(f"Total CO₂ emissions: {emissions:.6f} kg")
    return emissions


In [24]:

# Run both trainings and compare emissions

baseline_emissions = train_with_emissions(use_peft=False)
peft_emissions = train_with_emissions(use_peft=True)

print("\n EMISSIONS COMPARISON")
print(f"Full fine-tuning CO₂: {baseline_emissions:.6f} kg")
print(f"PEFT / LoRA CO₂    : {peft_emissions:.6f} kg")
if baseline_emissions > 0:
    reduction = (1 - peft_emissions / baseline_emissions) * 100
    print(f"Relative reduction  : {reduction:.2f}%")
else:
    print("Baseline emissions are zero or invalid; cannot compute reduction.")



 Starting experiment: full_finetune


Step,Training Loss
10,1.033100
20,0.973700
30,0.949500
40,1.036600
50,1.038500
60,1.011600
70,0.949400
80,0.952500
90,1.023900
100,1.001400


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...inetune/training_args.bin: 100%|##########| 5.84kB / 5.84kB            

...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...ft_lora/training_args.bin: 100%|##########| 5.84kB / 5.84kB            

  ...adapter_model.safetensors:  89%|########8 | 33.5MB / 37.8MB            

Training finished: peft_lora
Total CO₂ emissions: 0.005877 kg

 EMISSIONS COMPARISON
Full fine-tuning CO₂: 0.007711 kg
PEFT / LoRA CO₂    : 0.005877 kg
Relative reduction  : 23.78%


In [1]:
from transformers import pipeline


model_name = "ngbinetou/outputs_peft_lora"
pipe = pipeline("text-generation", model=model_name)


prompt = "Give me a Python code for factioral function."
result = pipe(prompt, max_length=100, pad_token_id=50259)


print(result[0]["generated_text"])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/968 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Give me a Python code for factioral function.


How to use Python's factorial operators?


```
def factorial(n): 
    if n == 0: 
        return 1
    return 2
    else: 
        return n * factorial(n-1) 

```

What can I do for it?


```
def factorial(n): 
    if n == 0: 
        return 1
    return 2
    else: 
        return n * factorial(n-1) 

```

How to use Python's factorial operators?


```
def factorial(n): 
    if n == 0: 
        return 1
    else: 
        return n * factorial(n-1) 

```

What can I do for it?


```
def factorial(n): 
    if n
